In [24]:
import os
import torch
import timm
import numpy as np
import pandas as pd
from PIL import Image, UnidentifiedImageError
from torchvision import transforms
from tqdm import tqdm

# --- Config ---
model_path = '/kaggle/working/best_convnext_model.pth'  # Updated model path
test_folder = '/kaggle/input/soil-classification/soil_classification-2025/test'
output_csv = 'submission_tta_convnext3.csv'
img_size = 224
class_names = ['Alluvial soil', 'Black Soil', 'Clay soil', 'Red soil']
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Base Transform ---
base_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# --- TTA Transforms ---
tta_transforms = [
    base_transform,  # original
    transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.RandomHorizontalFlip(p=1.0),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
    ]),
    transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.RandomVerticalFlip(p=1.0),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
    ]),
]

# --- Load Model (ConvNeXt Tiny) ---
model = timm.create_model('convnext_tiny', pretrained=False, num_classes=len(class_names))
try:
    model.load_state_dict(torch.load(model_path, map_location=device))
    print("✅ Model loaded successfully.")
except Exception as e:
    raise RuntimeError(f"❌ Failed to load model from {model_path}: {e}")

model.to(device)
model.eval()

# --- Prediction with TTA ---
results = []
valid_extensions = ('.jpg', '.jpeg', '.png', '.webp', '.gif')

for fname in tqdm(sorted(os.listdir(test_folder)), desc="🔍 Predicting with TTA"):
    if not fname.lower().endswith(valid_extensions):
        continue
    try:
        path = os.path.join(test_folder, fname)
        image = Image.open(path).convert('RGB')

        # TTA inference
        predictions = []
        for tta in tta_transforms:
            img_tensor = tta(image).unsqueeze(0).to(device)
            with torch.no_grad():
                output = model(img_tensor)
                predictions.append(torch.softmax(output, dim=1))

        avg_prediction = torch.mean(torch.stack(predictions), dim=0)
        pred_class = torch.argmax(avg_prediction, dim=1).item()

        results.append([fname, class_names[pred_class]])

    except UnidentifiedImageError:
        print(f"⚠️ Skipped unreadable image: {fname}")
    except Exception as e:
        print(f"❌ Error processing {fname}: {e}")

# --- Save Submission ---
df1 = pd.DataFrame(results, columns=['image_id', 'soil_type'])
df1.to_csv(output_csv, index=False)
print(f"\n✅ TTA-enhanced predictions saved to: {output_csv}")


✅ Model loaded successfully.


🔍 Predicting with TTA: 100%|██████████| 341/341 [00:11<00:00, 30.44it/s]


✅ TTA-enhanced predictions saved to: submission_tta_convnext3.csv
